Import modules from web app

In [1]:
import sys, os.path

modules_dir = os.path.join(os.path.abspath(''),"..") + "/main/modules"
sys.path.append(modules_dir)

import scraping

Retrieve HTML text from website

In [2]:
response = scraping.request("bbc.co.uk")

Check whether the request was successful

In [3]:
print(response.status_code)
print(response.text[0:100])

200
<!DOCTYPE html><html lang="en-GB" class="no-js"><head><meta charSet="utf-8" /><meta name="viewport" 


Read the HTML content as a string to analyse. In order for people to replicate my results, I've already downloaded the HTML text.

In [4]:
#response_text = response.text
#with open("response.txt", "w") as text_file:
#    text_file.write(response_text)

with open("response.txt","r") as text_file:
    response_text = text_file.read()

Process the HTML text into tokens, then sort so you can replicate my results

In [5]:
tokens = scraping.process(response_text)
tokens.sort()

Check the output...

In [6]:
print(len(tokens))
print(tokens[0:10])

73
[' 2021 BBC. The BBC is not responsible for the content of external sites. ', "'I'm just not ready to buy an electric car'", "'I've had to develop a way of getting results instantly'", "'One signing could decide the title race - but it's not Harry Kane'", "'One signing could decide the title race, but it's not Harry Kane'", "'We drove from Ireland to Australia in a camper van'", "'We had to tip milk down the drain - now we sell 200-400 bottles a day'", "'We never knew how dangerous Loch Lomond was'", "'When you're on a BMX 20 feet in the air there's no room for error'", "'When you're on a BMX 20 feet in the air there's no room for error' Video"]


Score each of the tokens using...
- [AFINN](https://github.com/fnielsen/afinn) = wordlist-based approach
- [VADER](https://github.com/cjhutto/vaderSentiment) = lexicon+rule-based approach

In [7]:
from afinn import Afinn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

afinn = Afinn()
vader = SentimentIntensityAnalyzer()
scored_tokens = []

for token in tokens:
  # afinn.score calculates by adding up individual scores for words
  # so you need to standardise by dividing the length of the phrase
  afinn_score = afinn.score(token) / len(token.split())

  vader_score = vader.polarity_scores(token)["compound"]
  
  scored_tokens.append({"token": token, "afinn_score": afinn_score, "vader_score": vader_score})

Check the output...

In [8]:
print(len(scored_tokens), scored_tokens[0:10])

73 [{'token': ' 2021 BBC. The BBC is not responsible for the content of external sites. ', 'afinn_score': 0.15384615384615385, 'vader_score': -0.2411}, {'token': "'I'm just not ready to buy an electric car'", 'afinn_score': 0.0, 'vader_score': -0.2755}, {'token': "'I've had to develop a way of getting results instantly'", 'afinn_score': 0.0, 'vader_score': 0.0}, {'token': "'One signing could decide the title race - but it's not Harry Kane'", 'afinn_score': 0.0, 'vader_score': 0.0}, {'token': "'One signing could decide the title race, but it's not Harry Kane'", 'afinn_score': 0.0, 'vader_score': 0.0}, {'token': "'We drove from Ireland to Australia in a camper van'", 'afinn_score': 0.0, 'vader_score': 0.0}, {'token': "'We had to tip milk down the drain - now we sell 200-400 bottles a day'", 'afinn_score': 0.0, 'vader_score': 0.0}, {'token': "'We never knew how dangerous Loch Lomond was'", 'afinn_score': -0.25, 'vader_score': 0.3724}, {'token': "'When you're on a BMX 20 feet in the air th

Let's move the data to pandas and spot-check some of the results

In [9]:
import pandas as pd

pd.set_option("max_row",None)
pd.set_option('display.max_colwidth', None)

token_df = pd.DataFrame(scored_tokens)

token_df[13:23]

,token,afinn_score,vader_score
13,Amazon moves Lord of the Rings production to UK,0.000000,0.1779
14,Britney Spears' father to step down as conservator,0.000000,0.0000
15,Casualty's 10 most memorable episodes,-0.200000,0.0000
16,"Composer, DJ and bandleader: The women redefining the sound of UK jazz",0.000000,0.0000
17,Create incredible pasta dishes with only five ingredients,0.000000,0.2732
18,Decades-old lesson found on hidden blackboard,0.000000,0.0000
19,England bat after Anderson takes five wickets in second Test,0.000000,0.0000
20,"England lose two wickets in two balls - clips, radio & text",0.000000,-0.4019
21,Ex-Olympian coached for years after abuse claim,-0.428571,-0.6369
22,Final call for all ballroom dancers!,0.000000,0.0000


Both scoring models seem pretty accurate and mostly agree with each other. The few exceptions I noticed when spot-checking:
- `Composer, DJ and bandleader...` should be a positive text, yet both models scored it as neutral. I can see why though, there weren't any particular words that suggested positivity.
- `Totally affordable...` was missed out by VADER as being a positive word

Let's now see all the places where the two models disagree

In [10]:
filtered_token = token_df[(token_df["afinn_score"] * token_df["vader_score"] <= 0) &
    ((token_df["afinn_score"] != 0) | (token_df["vader_score"] != 0))]

filtered_token.count()

token          14
afinn_score    14
vader_score    14
dtype: int64

So there's 14 out of 79 places where the 2 models disagree, which is roughly 25% of the time.

Let's have a look where exactly there's disagreements.

In [11]:
filtered_token

,token,afinn_score,vader_score
8,'I'm just not ready to buy an electric car',0.000000,-0.2755
14,2021 BBC. The BBC is not responsible for the content of external sites.,0.153846,-0.2411
19,Amazon moves Lord of the Rings production to UK,0.000000,0.1779
22,'We never knew how dangerous Loch Lomond was',-0.250000,0.3724
23,Why the Tourette's queen of Twitch hasn't been banned,-0.222222,0.3570
44,"England lose two wickets in two balls - clips, radio & text",0.000000,-0.4019
48,Women's Hundred: Trent Rockets struggle in must-win game against Birmingham Phoenix,0.181818,-0.3182
49,UK records a further 100 Covid deaths,-0.285714,0.0000
61,"Germany fears thousands got saline, not vaccine",0.000000,-0.4215
63,"Murder-accused boy, 14, in court after stabbing",-0.571429,0.0000


AFINN got 6 right, while VADER got the remainder 8. There's no clear winner then. What if we now look into this using a trained, state-of-the-art model? Enter HuggingFace.

We have several options on the menu: classic BERT, DistilBERT, RoBERTa and XLNet. I'm choosing DistilBERT because it's the most resource-light - this is a side project, after all.

,token,afinn_score,vader_score
33,Why you can trust the BBC,0.166667,0.5106
34,GB's Ujah suspended after positive test,0.166667,0.1280
35,'We never knew how dangerous Loch Lomond was',-0.250000,0.3724
36,2021 BBC. The BBC is not responsible for the content of external sites.,0.153846,-0.2411
37,"Woman arrested in murder probe after boy, 2, dies",-0.555556,-0.8316
38,Russia millionaire kills man he 'mistook for bear',-0.375000,-0.5423
39,Casualty's 10 most memorable episodes,-0.200000,0.0000
40,PM calls emergency meeting to discuss Afghanistan,-0.285714,-0.3818
41,"One mystery, four suspects, many lies",-0.166667,-0.6369
42,"Composer, DJ and bandleader: The women redefining the sound of UK jazz",0.000000,0.0000
